In [23]:
import os

In [24]:
import wandb



In [25]:
!wandb login

^C


In [4]:
import ultralytics
ultralytics.__version__

'8.3.23'

In [5]:
# Not training the model, just using pretrained model trained on COCO dataset. 
# Aquest dataset té som vehicles a part d'altres objectes.
#Fer servir aquest model per la detecció i després track i count
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import os


os.environ["YOLO_VERBOSE"] = "false"


model=YOLO('yolov5n.pt')

PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [6]:
#list of COCO dataset classes
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [8]:
videos = ["output2.mp4", "output3.mp4", "output5.mp4", "output6.mp4", "output7.mp4"]

## DETECCIÓ DE MOVIMENT

In [12]:

import time
import numpy as np



for video in videos:

    wandb.init(
    # set the wandb project where this run will be logged
    project="PSIV",
    name = video+" NOMES DETECCIO MOVIMENT",
    # track hyperparameters and run metadata
    config={
    "VIDEO": video
    }
)
    #creant una instancia de tracker que ve de from tracker import*
    tracker=Tracker()
    count=0
    cap=cv2.VideoCapture(video)

    #VARIABLES PER GUARDAR COTXES UP I DOWN
    down={}
    up={}

    counter_down=[]
    counter_up=[]
    downwards = 0
    upwards = 0
    line_y = 800  # Coordenada y de la línia única
    frames = 0
    current_fps = 0


    start_time = time.time()


    frame_difference = np.array([])

    pts = np.array([[359, 340],[186, 371] ,[8, 953], [446, 951]], np.int32)
    # divide all values by 2
    pts = pts.reshape((-1, 1, 2))

    last_frame = None

    while True:
        skip = False
        ret, frame = cap.read()
        if not ret:
            break
        count += 1
        # RESOLUCIO IMATGE ORIGINAL: 960X540
        # BAIXAR RESOLUCIO
        original_frame = frame.copy()

        mask = np.zeros_like(frame, dtype=np.uint8)
        cv2.fillPoly(mask, [pts], (255, 255, 255))
        
        if last_frame is not None:
            frame_difference = cv2.absdiff(last_frame, frame)*2


            frame_difference = cv2.bitwise_and(frame_difference, mask)


            # cv2.imshow("frame_difference", frame_difference)

        # Saltar 1 de cada 2 frames i quan la mascara no hi ha yolo
        if frame_difference.mean() < 1:
                cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # red line
                # if frame_difference.mean() > 1:
                #     cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0, 255), 2) 
                # cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                # cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                

        # Convertir el frame a grisos per anar més ràpid: 
        # No es pot utilitzar perquè YOLO treballa en RGB
        # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


        else:
            # YOLO Prediction
            results = model.predict(frame)

            # aconseguint les dades de la bounding box i confiança
            a = results[0].boxes.data
            a = a.detach().cpu().numpy()
            px = pd.DataFrame(a).astype("float")

            # only work with car class. only count cars. Quedarnos bounding box dels cars
            car_bboxes = []
            for index, row in px.iterrows():
                x1 = int(row[0])
                y1 = int(row[1])
                x2 = int(row[2])
                y2 = int(row[3])
                d = int(row[5])
                c = class_list[d]
                if 'car' in c:
                    car_bboxes.append([x1, y1, x2, y2])

            # provide les coordinades al tracker
            bbox_id = tracker.update(car_bboxes)

            # draw bbox
            for bbox in bbox_id:
                x3, y3, x4, y4, id = bbox
                cx = int((x3 + x4) // 2)
                cy = int((y3 + y4) // 2)

                # Actualitzar comptadors de cotxes que baixen o pugen
                offset = 7  # Margen d'error per la detecció de passos per la línia

                # Si el cotxe estava a la part superior i després passa a la part inferior, comptar com a baixada
                if cy < line_y - offset:  # Cotxe a la part superior de la línia
                    down[id] = cy  # Guardar la y del cotxe quan està a la part superior
                if id in down:
                    if cy > line_y + offset:  # Cotxe passa a la part inferior
                        # Dibuixar un punt vermell al cotxe que baixa
                        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                        # Mostrar l'ID del cotxe al frame
                        cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                        counter_down.append(id)  # Afegir l'ID a la llista de cotxes que baixen

                # Si el cotxe estava a la part inferior i després passa a la part superior, comptar com a pujada
                if cy > line_y + offset:  # Cotxe a la part inferior de la línia
                    up[id] = cy  # Guardar la y del cotxe quan està a la part inferior
                if id in up:
                    if cy < line_y - offset:  # Cotxe passa a la part superior
                        # Dibuixar un punt vermell al cotxe que puja
                        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                        # Mostrar l'ID del cotxe al frame
                        cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                        counter_up.append(id)  # Afegir l'ID a la llista de cotxes que pugen

                # Dibuixar la línia vermella al frame
            cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # línia vermella (única)
            # cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Show counters on the frame
        downwards = len(set(counter_down))
        upwards = len(set(counter_up))



        # FPS calculation
        #count the fps by counting the number of frames per second 
        frames += 1
        if frames == 20:
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time > 0:
                current_fps = 20/elapsed_time
            start_time = time.time()
            frames = 0

            wandb.log({"downwards": downwards, "upwards": upwards, "fps": current_fps, "YOLO_ACTIVAT": 1 if frame_difference.mean() > 1 else 0})

        # cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        # cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            
        cv2.imshow("frames", frame)

        
        if cv2.waitKey(1) & 0xFF == 27:
            break
        
        last_frame = original_frame
    wandb.finish()
    cap.release()
    cv2.destroyAllWindows()



0: 640x384 3 cars, 1 traffic light, 156.1ms
Speed: 8.0ms preprocess, 156.1ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 384)


C:\Users\polri\AppData\Local\Temp\ipykernel_25708\2181816694.py:68: RuntimeWarning: Mean of empty slice.
  if frame_difference.mean() < 1:
c:\Users\polri\Documents\00_Uni\4rt Curs - 2024_25\1r semestre\MAPSIV\01_Reptes\Repte2_Cotxes\PSIV_Repte2\venvBO\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



0: 640x384 4 cars, 125.4ms
Speed: 2.6ms preprocess, 125.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 cars, 101.3ms
Speed: 3.6ms preprocess, 101.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 4 cars, 1 traffic light, 106.2ms
Speed: 3.5ms preprocess, 106.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 4 cars, 101.6ms
Speed: 3.0ms preprocess, 101.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 107.9ms
Speed: 3.2ms preprocess, 107.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 cars, 94.7ms
Speed: 4.0ms preprocess, 94.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 cars, 1 traffic light, 107.5ms
Speed: 2.5ms preprocess, 107.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 1 traffic light, 100.1ms
Speed: 2.5ms preproce

YOLO_ACTIVAT,█▁▁▁▁▁█▁▁▁█▁▁▁█▁████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁
downwards,▁▁▁▁▁▁▁▁▁▁▁▁████████████████████████████
fps,▄▁▁▁▇█▇█▄▁▅▁▁▆▇▆▇▇▇▇▆▆▇▆▇▃▇▆▇▆▇▇█▁▁▄▁▄▁▁
upwards,▁▂▂▂▂▂▂▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▇▇███
YOLO_ACTIVAT,0
downwards,1
fps,38.35359
upwards,6


## MODEL INICIAL Yolov8

In [1]:


model = YOLO('yolov8m.pt')

for video in videos:

    wandb.init(
    # set the wandb project where this run will be logged
    project="PSIV",
    name = video + " INICIAL v8",
    # track hyperparameters and run metadata
    config={
    "VIDEO": video
    }
)
    #creant una instancia de tracker que ve de from tracker import*
    tracker=Tracker()
    count=0
    cap=cv2.VideoCapture(video)

    #VARIABLES PER GUARDAR COTXES UP I DOWN
    down={}
    up={}

    counter_down=[]
    counter_up=[]
    downwards = 0
    upwards = 0
    line_y = 800  # Coordenada y de la línia única
    frames = 0
    current_fps = 0


    start_time = time.time()

    last_frame = None

    while True:
        skip = False
        ret, frame = cap.read()
        if not ret:
            break
        count += 1
        # RESOLUCIO IMATGE ORIGINAL: 960X540
        # BAIXAR RESOLUCIO
        # frame = cv2.resize(frame, (270, 480))
        original_frame = frame.copy()

            # print(frame_difference.mean())

            # cv2.imshow("frame_difference", frame_difference)

        # Saltar 1 de cada 2 frames i quan la mascara no hi ha yolo
        # if count % 2 != 0 or frame_difference.mean() < 1:
                # cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # red line
                # if frame_difference.mean() > 1:
                #     cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0, 255), 2) 
                # cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                # cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                

        # Convertir el frame a grisos per anar més ràpid: 
        # No es pot utilitzar perquè YOLO treballa en RGB
        # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


        # else:
            # YOLO Prediction
        results = model.predict(frame)

        # aconseguint les dades de la bounding box i confiança
        a = results[0].boxes.data
        a = a.detach().cpu().numpy()
        px = pd.DataFrame(a).astype("float")

        # only work with car class. only count cars. Quedarnos bounding box dels cars
        car_bboxes = []
        for index, row in px.iterrows():
            x1 = int(row[0])
            y1 = int(row[1])
            x2 = int(row[2])
            y2 = int(row[3])
            d = int(row[5])
            c = class_list[d]
            if 'car' in c:
                car_bboxes.append([x1, y1, x2, y2])

        # provide les coordinades al tracker
        bbox_id = tracker.update(car_bboxes)

        # draw bbox
        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            cx = int((x3 + x4) // 2)
            cy = int((y3 + y4) // 2)

            # Actualitzar comptadors de cotxes que baixen o pugen
            offset = 7  # Margen d'error per la detecció de passos per la línia

            # Si el cotxe estava a la part superior i després passa a la part inferior, comptar com a baixada
            if cy < line_y - offset:  # Cotxe a la part superior de la línia
                down[id] = cy  # Guardar la y del cotxe quan està a la part superior
            if id in down:
                if cy > line_y + offset:  # Cotxe passa a la part inferior
                    # Dibuixar un punt vermell al cotxe que baixa
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                    # Mostrar l'ID del cotxe al frame
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_down.append(id)  # Afegir l'ID a la llista de cotxes que baixen

            # Si el cotxe estava a la part inferior i després passa a la part superior, comptar com a pujada
            if cy > line_y + offset:  # Cotxe a la part inferior de la línia
                up[id] = cy  # Guardar la y del cotxe quan està a la part inferior
            if id in up:
                if cy < line_y - offset:  # Cotxe passa a la part superior
                    # Dibuixar un punt vermell al cotxe que puja
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                    # Mostrar l'ID del cotxe al frame
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_up.append(id)  # Afegir l'ID a la llista de cotxes que pugen

            # Dibuixar la línia vermella al frame
        cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # línia vermella (única)
        # cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        downwards = len(set(counter_down))
        upwards = len(set(counter_up))



        # Contar frames cada 20 frames
        frames +=1
        if frames == 20:
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time > 0:
                current_fps = 20/elapsed_time
            start_time = time.time()
            frames = 0

            wandb.log({"downwards": downwards, "upwards": upwards, "fps": current_fps})

        # cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
        # cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        # cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                
        cv2.imshow("frames", frame)

            
        if cv2.waitKey(1) & 0xFF == 27:
            break
            
        last_frame = original_frame
    
    wandb.finish()
    cap.release()
    cv2.destroyAllWindows()


NameError: name 'YOLO' is not defined

## Model Inicial Baixa Resolució

In [31]:


for video in videos:

    wandb.init(
    # set the wandb project where this run will be logged
    project="PSIV",
    name = video+"INICIAL + MEITAT RESOLUCIO",
    # track hyperparameters and run metadata
    config={
    "VIDEO": video
    }
)
    #creant una instancia de tracker que ve de from tracker import*
    tracker=Tracker()
    count=0
    cap=cv2.VideoCapture(video)

    #VARIABLES PER GUARDAR COTXES UP I DOWN
    down={}
    up={}

    counter_down=[]
    counter_up=[]
    downwards = 0
    upwards = 0
    line_y = 400  # Coordenada y de la línia única
    frames = 0
    current_fps = 0


    start_time = time.time()

    last_frame = None

    while True:
        skip = False
        ret, frame = cap.read()
        if not ret:
            break
        count += 1
        # RESOLUCIO IMATGE ORIGINAL: 960X540
        # BAIXAR RESOLUCIO
        frame = cv2.resize(frame, (270, 480))
        original_frame = frame.copy()

            # print(frame_difference.mean())

            # cv2.imshow("frame_difference", frame_difference)

        # Saltar 1 de cada 2 frames i quan la mascara no hi ha yolo
        # if count % 2 != 0 or frame_difference.mean() < 1:
                # cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # red line
                # if frame_difference.mean() > 1:
                #     cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0, 255), 2) 
                # cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                # cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                

        # Convertir el frame a grisos per anar més ràpid: 
        # No es pot utilitzar perquè YOLO treballa en RGB
        # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


        # else:
            # YOLO Prediction
        results = model.predict(frame)

        # aconseguint les dades de la bounding box i confiança
        a = results[0].boxes.data
        a = a.detach().cpu().numpy()
        px = pd.DataFrame(a).astype("float")

        # only work with car class. only count cars. Quedarnos bounding box dels cars
        car_bboxes = []
        for index, row in px.iterrows():
            x1 = int(row[0])
            y1 = int(row[1])
            x2 = int(row[2])
            y2 = int(row[3])
            d = int(row[5])
            c = class_list[d]
            if 'car' in c:
                car_bboxes.append([x1, y1, x2, y2])

        # provide les coordinades al tracker
        bbox_id = tracker.update(car_bboxes)

        # draw bbox
        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            cx = int((x3 + x4) // 2)
            cy = int((y3 + y4) // 2)

            # Actualitzar comptadors de cotxes que baixen o pugen
            offset = 7  # Margen d'error per la detecció de passos per la línia

            # Si el cotxe estava a la part superior i després passa a la part inferior, comptar com a baixada
            if cy < line_y - offset:  # Cotxe a la part superior de la línia
                down[id] = cy  # Guardar la y del cotxe quan està a la part superior
            if id in down:
                if cy > line_y + offset:  # Cotxe passa a la part inferior
                    # Dibuixar un punt vermell al cotxe que baixa
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                    # Mostrar l'ID del cotxe al frame
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_down.append(id)  # Afegir l'ID a la llista de cotxes que baixen

            # Si el cotxe estava a la part inferior i després passa a la part superior, comptar com a pujada
            if cy > line_y + offset:  # Cotxe a la part inferior de la línia
                up[id] = cy  # Guardar la y del cotxe quan està a la part inferior
            if id in up:
                if cy < line_y - offset:  # Cotxe passa a la part superior
                    # Dibuixar un punt vermell al cotxe que puja
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                    # Mostrar l'ID del cotxe al frame
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_up.append(id)  # Afegir l'ID a la llista de cotxes que pugen

            # Dibuixar la línia vermella al frame
        cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # línia vermella (única)
        # cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Show counters on the frame
        downwards = len(set(counter_down))
        upwards = len(set(counter_up))



        frames +=1
        if frames == 20:
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time > 0:
                current_fps = 20/elapsed_time
            start_time = time.time()
            frames = 0

            wandb.log({"downwards": downwards, "upwards": upwards, "fps": current_fps})

        cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
        cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                
        cv2.imshow("frames", frame)

            
        if cv2.waitKey(1) & 0xFF == 27:
            break
            
        last_frame = original_frame
    
    wandb.finish()
    cap.release()
    cv2.destroyAllWindows()



0: 640x384 3 cars, 1 traffic light, 103.2ms
Speed: 4.0ms preprocess, 103.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 79.4ms
Speed: 2.0ms preprocess, 79.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 86.7ms
Speed: 2.0ms preprocess, 86.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 91.4ms
Speed: 3.2ms preprocess, 91.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 91.8ms
Speed: 2.6ms preprocess, 91.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 84.7ms
Speed: 3.3ms preprocess, 84.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 86.9ms
Speed: 2.5ms preprocess, 86.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384

downwards,▁▁▁▁▁▁▁▁▁▅▅█████████████████████████████
fps,██▆▆█▄▆▇▆▆▄▄▁▇▅▆▇▇▆▆█▅▅▇█▂▆▄█▇▆▃▄▆▆▆▆▆▆▆
upwards,▁▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆█████
downwards,2
fps,8.21006
upwards,6


## Model Inicial cada 2 frames

In [30]:



for video in videos:

    wandb.init(
    # set the wandb project where this run will be logged
    project="PSIV",
    name = video+"INICIAL + SALTEM 1 DE CADA 2",
    # track hyperparameters and run metadata
    config={
    "VIDEO": video
    }
)
    #creant una instancia de tracker que ve de from tracker import*
    tracker=Tracker()
    count=0
    cap=cv2.VideoCapture(video)

    #VARIABLES PER GUARDAR COTXES UP I DOWN
    down={}
    up={}

    counter_down=[]
    counter_up=[]
    downwards = 0
    upwards = 0
    line_y = 800  # Coordenada y de la línia única
    frames = 0
    current_fps = 0


    start_time = time.time()

    last_frame = None

    while True:
        skip = False
        ret, frame = cap.read()
        if not ret:
            break
        count += 1
        # RESOLUCIO IMATGE ORIGINAL: 960X540
        # BAIXAR RESOLUCIO
        # frame = cv2.resize(frame, (270, 480))
        original_frame = frame.copy()

            # print(frame_difference.mean())

            # cv2.imshow("frame_difference", frame_difference)

        # Saltar 1 de cada 2 frames i quan la mascara no hi ha yolo
        if count % 2 != 0:
                cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # red line
                # if frame_difference.mean() > 1:
                #     cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0, 255), 2) 
                # cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                # cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                

        # Convertir el frame a grisos per anar més ràpid: 
        # No es pot utilitzar perquè YOLO treballa en RGB
        # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


        else:
                # YOLO Prediction
            results = model.predict(frame)

            # aconseguint les dades de la bounding box i confiança
            a = results[0].boxes.data
            a = a.detach().cpu().numpy()
            px = pd.DataFrame(a).astype("float")

            # only work with car class. only count cars. Quedarnos bounding box dels cars
            car_bboxes = []
            for index, row in px.iterrows():
                x1 = int(row[0])
                y1 = int(row[1])
                x2 = int(row[2])
                y2 = int(row[3])
                d = int(row[5])
                c = class_list[d]
                if 'car' in c:
                    car_bboxes.append([x1, y1, x2, y2])

            # provide les coordinades al tracker
            bbox_id = tracker.update(car_bboxes)

            # draw bbox
            for bbox in bbox_id:
                x3, y3, x4, y4, id = bbox
                cx = int((x3 + x4) // 2)
                cy = int((y3 + y4) // 2)

                # Actualitzar comptadors de cotxes que baixen o pugen
                offset = 7  # Margen d'error per la detecció de passos per la línia

                # Si el cotxe estava a la part superior i després passa a la part inferior, comptar com a baixada
                if cy < line_y - offset:  # Cotxe a la part superior de la línia
                    down[id] = cy  # Guardar la y del cotxe quan està a la part superior
                if id in down:
                    if cy > line_y + offset:  # Cotxe passa a la part inferior
                        # Dibuixar un punt vermell al cotxe que baixa
                        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                        # Mostrar l'ID del cotxe al frame
                        cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                        counter_down.append(id)  # Afegir l'ID a la llista de cotxes que baixen

                # Si el cotxe estava a la part inferior i després passa a la part superior, comptar com a pujada
                if cy > line_y + offset:  # Cotxe a la part inferior de la línia
                    up[id] = cy  # Guardar la y del cotxe quan està a la part inferior
                if id in up:
                    if cy < line_y - offset:  # Cotxe passa a la part superior
                        # Dibuixar un punt vermell al cotxe que puja
                        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                        # Mostrar l'ID del cotxe al frame
                        cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                        counter_up.append(id)  # Afegir l'ID a la llista de cotxes que pugen

                # Dibuixar la línia vermella al frame
            cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # línia vermella (única)
        # cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Show counters on the frame
        downwards = len(set(counter_down))
        upwards = len(set(counter_up))



    # FPS calculation
    #count the fps by counting the number of frames per second 
        frames +=1
        if frames == 20:
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time > 0:
                current_fps = 20/elapsed_time
            start_time = time.time()
            frames = 0

            wandb.log({"downwards": downwards, "upwards": upwards, "fps": current_fps})

        cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
        cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                
        cv2.imshow("frames", frame)

            
        if cv2.waitKey(1) & 0xFF == 27:
            break
            
        last_frame = original_frame
    
    wandb.finish()
    cap.release()
    cv2.destroyAllWindows()


downwards,▁▁
fps,▁█
upwards,▁▁
downwards,0
fps,10.65034
upwards,0



0: 640x384 3 cars, 1 traffic light, 90.4ms
Speed: 3.0ms preprocess, 90.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 89.0ms
Speed: 2.5ms preprocess, 89.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 84.4ms
Speed: 2.0ms preprocess, 84.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 90.7ms
Speed: 2.0ms preprocess, 90.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 86.1ms
Speed: 2.5ms preprocess, 86.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 95.1ms
Speed: 1.7ms preprocess, 95.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cars, 1 traffic light, 98.1ms
Speed: 2.0ms preprocess, 98.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3

downwards,▁▁▁▁▁▁▁▁▁▅▅▅▅▅██████████████████████████
fps,▆▆▆▆█▅▆▆▆▅▃▃▄▄▄▅▃▃▄▄▅▅▄▄▁▄▂▃▄▄▃▄▄▄▅▅▅▄▄▅
upwards,▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▇█
downwards,2
fps,15.57689
upwards,6


## Model ROI

In [42]:
import time
import numpy as np



for video in videos:

    wandb.init(
    # set the wandb project where this run will be logged
    project="PSIV",
    name = video+"INICIAL + ROI",
    # track hyperparameters and run metadata
    config={
    "VIDEO": video
    }
)
    #creant una instancia de tracker que ve de from tracker import*
    tracker=Tracker()
    count=0
    cap=cv2.VideoCapture(video)

    #VARIABLES PER GUARDAR COTXES UP I DOWN
    down={}
    up={}

    counter_down=[]
    counter_up=[]
    downwards = 0
    upwards = 0
    line_y = 400  # Coordenada y de la línia única
    frames = 0
    current_fps = 0



# Inicialitzar el comptador de FPS
    start_time = time.time()
    pts = np.array([[359, 480],[186, 480] ,[8, 840], [446, 841]], np.int32)
    # divide all values by 2
    pts = pts.reshape((-1, 1, 2))
    pts = pts // 2

    # Bucle infinit per processar el vídeo frame a frame
    while True:
        # Llegir un nou frame de la captura de vídeo
        ret, frame = cap.read()
        if not ret:  # Si no hi ha més frames, surt del bucle
            break
        count += 1

        # RESOLUCIÓ IMATGE ORIGINAL: 960X540
        # REDUIR RESOLUCIÓ a 270x480
        frame = cv2.resize(frame, (270, 480))
        original_frame = frame.copy()

        mask = np.zeros_like(frame, dtype=np.uint8)
        cv2.fillPoly(mask, [pts], (255, 255, 255))
        roi_frame = cv2.bitwise_and(original_frame, mask)
        

        results = model.predict(roi_frame)

        # Obtenir les dades de la bounding box i la confiança de la predicció
        a = results[0].boxes.data
        a = a.detach().cpu().numpy()
        px = pd.DataFrame(a).astype("float")

        # Filtrar només les bounding boxes de cotxes
        car_bboxes = []
        for index, row in px.iterrows():
            x1 = int(row[0])
            y1 = int(row[1])
            x2 = int(row[2])
            y2 = int(row[3])
            d = int(row[5])
            c = class_list[d]  # Obtenir la classe de l'objecte detectat
            if 'car' in c:  # Si l'objecte és un cotxe, guardar la seva bounding box
                car_bboxes.append([x1, y1, x2, y2])

        # Proporcionar les coordenades de les bounding boxes al tracker
        bbox_id = tracker.update(car_bboxes)

        # Dibuixar les bounding boxes i els IDs al frame
        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            cx = int((x3 + x4) // 2)  # Coordenada central de la bounding box (x)
            cy = int((y3 + y4) // 2)  # Coordenada central de la bounding box (y)

            # Actualitzar comptadors de cotxes que baixen o pugen
            line_y = 400  # Coordenada y de la línia única
            offset = 7  # Margen d'error per la detecció de passos per la línia

            # Si el cotxe estava a la part superior i després passa a la part inferior, comptar com a baixada
            if cy < line_y - offset:  # Cotxe a la part superior de la línia
                down[id] = cy  # Guardar la y del cotxe quan està a la part superior
            if id in down:
                if cy > line_y + offset:  # Cotxe passa a la part inferior
                    # Dibuixar un punt vermell al cotxe que baixa
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                    # Mostrar l'ID del cotxe al frame
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_down.append(id)  # Afegir l'ID a la llista de cotxes que baixen

            # Si el cotxe estava a la part inferior i després passa a la part superior, comptar com a pujada
            if cy > line_y + offset:  # Cotxe a la part inferior de la línia
                up[id] = cy  # Guardar la y del cotxe quan està a la part inferior
            if id in up:
                if cy < line_y - offset:  # Cotxe passa a la part superior
                    # Dibuixar un punt vermell al cotxe que puja
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                    # Mostrar l'ID del cotxe al frame
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_up.append(id)  # Afegir l'ID a la llista de cotxes que pugen

        # Dibuixar la línia vermella al frame
        cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # línia vermella (única)

        # Mostrar el nombre de cotxes que han baixat i pujat
        downwards = len(set(counter_down))  # Comptar cotxes únics que han baixat
        upwards = len(set(counter_up))  # Comptar cotxes únics que han pujat

        # Imprimir els resultats
        print(f"Baixada: {downwards}")
        print(f"Pujada: {upwards}")

        # Càlcul de FPS (frames per segon)
        frames +=1
        if frames == 20:
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time > 0:
                current_fps = 20/elapsed_time
            start_time = time.time()
            frames = 0
            wandb.log({"downwards": downwards, "upwards": upwards, "fps": current_fps})

        cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Mostrar el frame actualitzat
        cv2.imshow("frames", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # Sortir si es prem la tecla ESC
            break

    # Alliberar la captura de vídeo i destruir totes les finestres

wandb.finish()
cap.release()
cv2.destroyAllWindows()


downwards,▁▁▁▁
fps,▁▂▇█
upwards,▁▁▁▁
downwards,0
fps,2.48627
upwards,0



0: 640x384 1 book, 389.6ms
Speed: 3.7ms preprocess, 389.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 1 book, 403.4ms
Speed: 2.0ms preprocess, 403.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 1 book, 422.1ms
Speed: 2.2ms preprocess, 422.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 1 book, 437.2ms
Speed: 3.0ms preprocess, 437.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 1 book, 470.9ms
Speed: 3.6ms preprocess, 470.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 1 book, 454.2ms
Speed: 3.5ms preprocess, 454.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
Baixada: 0
Pujada: 0

0: 640x384 1 book, 433.5ms
Speed: 3.7ms preprocess, 433.5ms inference, 1.0ms postprocess per image at shape (1, 3, 64

downwards,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fps,▄▄▃▄▃▃▆██▇▇▆█▇▆█▇▇▇▇█▆▁▇▆▆▇▇▇▇▇▇▇▇▇▇▇▆▆▆
upwards,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
downwards,0
fps,2.54152
upwards,0


## Model Final

In [32]:




import time
import numpy as np


for video in videos:

    wandb.init(
    # set the wandb project where this run will be logged
    project="PSIV",
    name = video+" MODELFINAL",
    # track hyperparameters and run metadata
    config={
    "VIDEO": video
    }
)
    #creant una instancia de tracker que ve de from tracker import*
    tracker=Tracker()
    count=0
    cap=cv2.VideoCapture(video)

    #VARIABLES PER GUARDAR COTXES UP I DOWN
    down={}
    up={}

    counter_down=[]
    counter_up=[]
    downwards = 0
    upwards = 0
    line_y = 400  # Coordenada y de la línia única
    frames = 0
    current_fps = 0


    start_time = time.time()
    pts_ROI = np.array([[359, 480],[186, 480] ,[8, 840], [446, 841]], np.int32)
    pts_ROI = pts_ROI // 2
    pts_ROI = pts_ROI.reshape((-1, 1, 2))

    frame_difference = np.array([])

    pts = np.array([[359, 340],[186, 371] ,[8, 953], [446, 951]], np.int32)
    # divide all values by 2
    pts = pts // 2
    pts = pts.reshape((-1, 1, 2))

    last_frame = None

    while True:
        skip = False
        ret, frame = cap.read()
        if not ret:
            break
        count += 1
        # RESOLUCIO IMATGE ORIGINAL: 960X540
        # BAIXAR RESOLUCIO
        frame = cv2.resize(frame, (270, 480))
        original_frame = frame.copy()

        mask = np.zeros_like(frame, dtype=np.uint8)
        cv2.fillPoly(mask, [pts], (255, 255, 255))
        roi_frame = cv2.bitwise_and(original_frame, mask)
        
        if last_frame is not None:
            frame_difference = cv2.absdiff(last_frame, frame)*2


            frame_difference = cv2.bitwise_and(frame_difference, mask)

            # print(frame_difference.mean())

            # cv2.imshow("frame_difference", frame_difference)

        # Saltar 1 de cada 2 frames i quan la mascara no hi ha yolo
        if count % 2 != 0 or frame_difference.mean() < 1:
                cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # red line
                if frame_difference.mean() > 1:
                    cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0, 255), 2) 
                # cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                # cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                

        # Convertir el frame a grisos per anar més ràpid: 
        # No es pot utilitzar perquè YOLO treballa en RGB
        # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


        else:
            # YOLO Prediction
            results = model.predict(roi_frame)

            # aconseguint les dades de la bounding box i confiança
            a = results[0].boxes.data
            a = a.detach().cpu().numpy()
            px = pd.DataFrame(a).astype("float")

            # only work with car class. only count cars. Quedarnos bounding box dels cars
            car_bboxes = []
            for index, row in px.iterrows():
                x1 = int(row[0])
                y1 = int(row[1])
                x2 = int(row[2])
                y2 = int(row[3])
                d = int(row[5])
                c = class_list[d]
                if 'car' in c:
                    car_bboxes.append([x1, y1, x2, y2])

            # provide les coordinades al tracker
            bbox_id = tracker.update(car_bboxes)

            # draw bbox
            for bbox in bbox_id:
                x3, y3, x4, y4, id = bbox
                cx = int((x3 + x4) // 2)
                cy = int((y3 + y4) // 2)

                # Actualitzar comptadors de cotxes que baixen o pugen
                offset = 7  # Margen d'error per la detecció de passos per la línia

                # Si el cotxe estava a la part superior i després passa a la part inferior, comptar com a baixada
                if cy < line_y - offset:  # Cotxe a la part superior de la línia
                    down[id] = cy  # Guardar la y del cotxe quan està a la part superior
                if id in down:
                    if cy > line_y + offset:  # Cotxe passa a la part inferior
                        # Dibuixar un punt vermell al cotxe que baixa
                        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                        # Mostrar l'ID del cotxe al frame
                        cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                        counter_down.append(id)  # Afegir l'ID a la llista de cotxes que baixen

                # Si el cotxe estava a la part inferior i després passa a la part superior, comptar com a pujada
                if cy > line_y + offset:  # Cotxe a la part inferior de la línia
                    up[id] = cy  # Guardar la y del cotxe quan està a la part inferior
                if id in up:
                    if cy < line_y - offset:  # Cotxe passa a la part superior
                        # Dibuixar un punt vermell al cotxe que puja
                        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                        # Mostrar l'ID del cotxe al frame
                        cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                        counter_up.append(id)  # Afegir l'ID a la llista de cotxes que pugen

                # Dibuixar la línia vermella al frame
            cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # línia vermella (única)
            cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Show counters on the frame
        downwards = len(set(counter_down))
        upwards = len(set(counter_up))



        # FPS calculation
        #count the fps by counting the number of frames per second 
        frames += 1
        if frames == 20:
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time > 0:
                current_fps = 20/elapsed_time
            start_time = time.time()
            frames = 0

            wandb.log({"downwards": downwards, "upwards": upwards, "fps": current_fps, "YOLO_ACTIVAT": 1 if frame_difference.mean() > 1 else 0})

        cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            
        cv2.imshow("frames", frame)

        
        if cv2.waitKey(1) & 0xFF == 27:
            break
        
        last_frame = original_frame
    wandb.finish()
    cap.release()
    cv2.destroyAllWindows()



0: 640x384 1 car, 83.3ms
Speed: 1.0ms preprocess, 83.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)


C:\Users\polri\AppData\Local\Temp\ipykernel_25708\3721818295.py:77: RuntimeWarning: Mean of empty slice.
  if frame_difference.mean() > 1:
c:\Users\polri\Documents\00_Uni\4rt Curs - 2024_25\1r semestre\MAPSIV\01_Reptes\Repte2_Cotxes\PSIV_Repte2\venvBO\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



0: 640x384 1 car, 77.6ms
Speed: 1.6ms preprocess, 77.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 78.9ms
Speed: 2.0ms preprocess, 78.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 72.2ms
Speed: 2.3ms preprocess, 72.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 77.4ms
Speed: 1.2ms preprocess, 77.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 68.0ms
Speed: 3.0ms preprocess, 68.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 76.8ms
Speed: 3.5ms preprocess, 76.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 74.5ms
Speed: 1.7ms preprocess, 74.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 76.0ms
Speed: 3.0ms preprocess, 76.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 ca

YOLO_ACTIVAT,▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
downwards,▁▂▂▂▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▆▆▆▇▇▇▇▇▇▇███████████
fps,▁▄▁▆▅▆▆▅▁▅▆▆▆▆▄▅▄▆▆▆▆▇▆▆▆▆▆▆▅▄▆█▆▆▆▆▆▇▇▅
upwards,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅█
YOLO_ACTIVAT,0
downwards,10
fps,27.86383
upwards,3
